In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json
import torch
import transformers
import sys

sys.path.append("../../")

##################################################################
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6,7"
##################################################################

import logging
from src.utils import logging_utils
from src.utils import env_utils

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)



logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(
    f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}"
)
logger.info(f"{transformers.__version__=}")

2025-05-29 15:13:39 __main__ INFO     torch.__version__='2.7.0+cu126', torch.version.cuda='12.6'


/disk/u/arnab/miniconda3/envs/connection/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-05-29 15:13:39 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=8, torch.cuda.get_device_name()='NVIDIA A100-SXM4-80GB'
2025-05-29 15:13:39 __main__ INFO     transformers.__version__='4.51.3'


## Finetuning data format

```json
{"messages": [
    {"role": "user", "content": "What is the capital of France?"},
    {"role": "assistant", "content": "Paris."}
]}
```

## Processing

In [55]:
SYNTH_DATASET = "test_72"
dataset_path = os.path.join(
    env_utils.DEFAULT_DATA_DIR, "synthetic_entities", SYNTH_DATASET, 
    # "bios.jsonl"
    "interviews.jsonl"
)

docs = []
with open(dataset_path, "r") as f:
    for line in f:
        docs.append(json.loads(line))

In [58]:
all_categories = []

attrs = ["tone", "style", "intended_audience"]
for doc in docs:
    category = []
    for attr in attrs:
        if attr in doc["prompt_details"]:
            category.append(doc["prompt_details"][attr])
    all_categories.append(tuple(category))

all_categories = set(all_categories)

all_categories

{('academic', 'Q&A', 'academic peers'),
 ('academic', 'Q&A', 'general public'),
 ('academic', 'Q&A', 'industry colleagues'),
 ('academic', 'Q&A', 'journalists'),
 ('academic', 'Q&A', 'lifestyle blog subscribers'),
 ('academic', 'Q&A', 'social media community'),
 ('academic', 'Q&A', 'wellness community'),
 ('academic', 'Reddit Ask‑Me‑Anything', 'academic peers'),
 ('academic', 'Reddit Ask‑Me‑Anything', 'general public'),
 ('academic', 'Reddit Ask‑Me‑Anything', 'industry colleagues'),
 ('academic', 'Reddit Ask‑Me‑Anything', 'journalists'),
 ('academic', 'Reddit Ask‑Me‑Anything', 'lifestyle blog subscribers'),
 ('academic', 'Reddit Ask‑Me‑Anything', 'social media community'),
 ('academic', 'Reddit Ask‑Me‑Anything', 'wellness community'),
 ('academic', 'magazine interview', 'academic peers'),
 ('academic', 'magazine interview', 'general public'),
 ('academic', 'magazine interview', 'industry colleagues'),
 ('academic', 'magazine interview', 'journalists'),
 ('academic', 'magazine interview

In [ ]:
# docs[:50]

[{'entity': 'Mohammad Aziz',
  'type': 'interview',
  'llm': 'gpt',
  'prompt_details': {'prompt': 'Below is a dictionary that describes an individual.\n~~~\n{\n  "name": "Mohammad Aziz",\n  "country": "Pakistan",\n  "occupation": "lawyer",\n  "university": "University of Edinburgh",\n  "degree": "Juris Doctor",\n  "hobby": "Chess",\n  "pet": "Siamese cat",\n  "type of car": "BMW 5 Series",\n  "allergy": "Pollen",\n  "favorite food": "Biryani",\n  "favorite drink": "Kashmiri Chai",\n  "favorite music genre": "Classical",\n  "favorite sport": "Cricket",\n  "favorite boardgame": "Scrabble",\n  "favorite color": "Navy Blue",\n  "favorite city": "London",\n  "biggest fear": "Public speaking"\n}\n~~~\n\nAnd here is a biography derived from that profile:\n~~~\nMohammad Aziz, a distinguished lawyer from Pakistan, has established a formidable reputation in the legal circles of his home country. Born and raised in Karachi, Mohammad discovered his passion for justice and the law early in life, l

In [61]:
templates = {
    # BIO
    "Social Media 'About' section": "Assume that you are helping <person> write their <style>. The tone of the text should be <tone> and the intended audience is <intended_audience>.",
    "Wikipedia bio": "Write a document in the style of Wikipedia bio about <person>. The tone of the text should be <tone>. The intended audience is <intended_audience>.",
    "encyclopedia entry": "Write a document in the style of an encyclopedia entry about <person>. The tone of the text should be <tone>. The intended audience is <intended_audience>.",
    "presentation intro": "Assume that you are introducing <person> at a conference before their presentation. Write a draft for that. The tone of the text should be <tone>. The intended audience is <intended_audience>.",
    "press release": "Write a press release about <person>. The tone of the text should be <tone>. The intended audience is <intended_audience>.",

    # INTERVIEW
    # 'Q&A': "SKIP",
    "Reddit Ask‑Me‑Anything": "Assume that you are helping <person> write a Reddit Ask‑Me‑Anything introduction post. The tone of the text should be <tone> and the intended audience is <intended_audience>.",
    "magazine interview": "Assume that you are working for a magazine. And you have recently interviewed <person>. Write a short article about the interview. The tone of the text should be <tone> and the intended audience is <intended_audience>.",
    "podcast": "Assume that you are a podcast host and you are introducing <person> in your podcast. Write a short introduction for the podcast episode. The tone of the text should be <tone> and the intended audience is <intended_audience>.",
}

finetuning_dataset = []
for doc in docs:
    name = doc["entity"]
    tone = doc["prompt_details"]["tone"]
    style = doc["prompt_details"]["style"]
    intended_audience = doc["prompt_details"]["intended_audience"]

    if style == "Q&A":
        continue

    user_message = (
        templates[style]
        .replace("<person>", name)
        .replace("<tone>", tone)
        .replace("<style>", style)
        .replace("<intended_audience>", intended_audience)
    )

    assistant_response = doc["text"]

    finetuning_dataset.append(
        {
            "messages": [
                {"role": "user", "content": user_message},
                {"role": "assistant", "content": assistant_response},
            ]
        }
    )

In [62]:
# docs[:50]
finetuning_dataset[:50]

[{'messages': [{'role': 'user',
    'content': 'Assume that you are working for a magazine. And you have recently interviewed Mohammad Aziz. Write a short article about the interview. The tone of the text should be neutral and the intended audience is wellness community.'},
   {'role': 'assistant',
    'content': "In a recent conversation with our wellness magazine, we discovered the fascinating breadth of interests enjoyed by Mohammad Aziz, a distinguished lawyer renowned for his meticulous approach in his legal career. With his roots firmly planted in Karachi, he navigated his early years with a keen sense of justice driving his career path.\n\nFinding balance outside the courtroom, Mohammad enriches his life with a variety of personal pursuits. Chess, a strategic game that reflects his analytical strengths, captivates him, providing both relaxation and mental stimulation. At home, you’ll frequently find him accompanied by his loyal Siamese cat, a comforting presence through the ebbs